In [1]:
from lib2to3.pytree import WildcardPattern
from xml.dom import minidom
from dataclasses import dataclass
import json
import lxml
from bs4 import BeautifulSoup, SoupStrainer
from xml.dom import minidom
import os
import argparse
import bz2
import logging
import os.path
import re  # TODO use regex when it will be standard
import sys
from io import StringIO
from multiprocessing import Queue, get_context, cpu_count
from timeit import default_timer
import re
import pymorphy2
from functools import lru_cache
from tqdm import tqdm
import pickle
from ruwordnet import RuWordNet
from collections import defaultdict
import time
from  extractor import Extractor
import numpy as np

In [3]:
acceptedNamespaces = ['w', 'wiktionary', 'wikt']
templateNamespace = ''
morph_analizer = pymorphy2.MorphAnalyzer()
tagRE = re.compile(r'(.*?)<(/?\w+)[^>]*>(?:([^<]*)(<.*?>)?)?')
def collect_pages(text):
    """
    :param text: the text of a wikipedia file dump.
    """
    # we collect individual lines, since str.join() is significantly faster
    # than concatenation
    page = []
    id = ''
    revid = ''
    last_id = ''
    inText = False
    redirect = False
    redirect_page = ''
    for line in text:
        if '<' not in line:     # faster than doing re.search()
            if inText:
                page.append(line)
            continue
        m = tagRE.search(line)
        if not m:
            continue
        tag = m.group(2)
        if tag == 'page':
            page = []
            redirect = False
        elif tag == 'id' and not id:
            id = m.group(3)
        elif tag == 'id' and id: # <revision> <id></id> </revision>
            revid = m.group(3)
        elif tag == 'title':
            title = m.group(3)
        elif tag == 'redirect':
            redirect = True
            redirectRE = re.compile(r'title=\"(.*?)\" />')
            redirect_page = re.findall(redirectRE, line)[0]
        elif tag == 'text':
            inText = True
            line = line[m.start(3):m.end(3)]
            page.append(line)
            if m.lastindex == 4:  # open-close
                inText = False
        elif tag == '/text':
            if m.group(1):
                page.append(m.group(1))
            inText = False
        elif inText:
            page.append(line)
        elif tag == '/page':
            colon = title.find(':')
            if (colon < 0 or (title[:colon] in acceptedNamespaces) and id != last_id and
                    not redirect and not title.startswith(templateNamespace)):
                yield (id, revid, title, page, redirect_page,redirect)
                last_id = id
            id = ''
            revid = ''
            page = []
            inText = False
            redirect = False
            redirect_page=''

def decode_open(filename, mode='rt', encoding='utf-8'):
    """
    Open a file, decode and decompress, depending on extension `gz`, or 'bz2`.
    :param filename: the file to open.
    """
    ext = os.path.splitext(filename)[1]
    if ext == '.gz':
        import gzip
        return gzip.open(filename, mode, encoding=encoding)
    elif ext == '.bz2':
        return bz2.open(filename, mode=mode, encoding=encoding)
    else:
        return open(filename, mode, encoding=encoding)

@lru_cache(maxsize=200000)
def get_normal_form(word):
    return morph_analizer.parse(word)[0].normal_form
wn = RuWordNet(filename_or_session='D:\\lbase_data\\ruwordnet.db')

In [4]:
@dataclass
class display:
    id:int
    revid:int
    title:str
    lemma:str
    wordId:int
    ctxW:set
class displayEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, display):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)

In [5]:
file = open("D:\\asd\\dictDisplay4.txt", "rb")
unpickler = pickle.Unpickler(file)
dictDisplay = unpickler.load()
file.close()

In [43]:
for key, value in dictDisplay.items():
    print(key, value)
    break


Единицы измерения давления display(id='7827', revid='6850', title='Единицы измерения давления', lemma='ЕДИНИЦА ИЗМЕРЕНИЕ ДАВЛЕНИЕ', wordId='138635-N', ctxW={'стена', 'миллиметр ртутный столб', 'микрон', 'барий', 'пьеза', 'единица измерение давление', 'давление', 'единица измерение', 'миллиметр водяной столб'})


In [6]:
@dataclass
class connection_wiki:
    id:int
    revid:int
    title:str
    lemma:str
    wordId:int
    text:str
    clear_text:str
    summary:str
    clear_summary:str
class connection_wikiEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, connection_wiki):
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)

In [7]:
matcher=re.compile(r"[A-Яа-яё]+-?[A-Яа-яё]")
def extract_clear_text(text):
    """извлечение  слов из текста и приведение их к нормальному виду
    вовзраащемое значение строка из нормализированных слов(знаки препинания удалены)"""
    return " ".join([get_normal_form(x) for x in matcher.findall(text)])

In [50]:
input = decode_open("D:\\asd\\ruwiki-20220701-pages-meta-current.xml.bz2")
i = 0
dict_lemma_in_text = {}
for id, revid, title, page, redirect_page, redirect in tqdm(collect_pages(input)):
    # print(title, "aaaa",''.join(page), file = printt)
    if title in dictDisplay:
        text = ''.join(page)
        ext = Extractor(id,revid,"",title,page)
        text = "\n".join(ext.clean_text(text))
        summary = text.split("\n")[0]
        dict_lemma_in_text[dictDisplay[title].lemma] = connection_wiki(id,revid,title, dictDisplay[title].wordId,dictDisplay[title].lemma,
                                                                    text, extract_clear_text(text), summary, extract_clear_text(summary))
        if get_normal_form(title) == "":
            break
input.close()

4432391it [20:14, 3649.98it/s] 


In [51]:
# сохранение dict_lemma_in_text
# file = open("dict_lemma_in_text.txt", "wb")
# pickle.dump(dict_lemma_in_text, file=file)
# file.close()

In [8]:
# считывание-+ dict_lemma_in_text
file = open("D:\\asd\\dict_lemma_in_text.txt", "rb")
unpickler = pickle.Unpickler(file)
dict_lemma_in_text = unpickler.load()
file.close()

In [12]:
for key in dict_lemma_in_text:
    print(dict_lemma_in_text[key].text.split(".")[0])
    break

Литва́ ( ), официальное название — Лито́вская Респу́блика () — государство, расположенное в северной Европе


In [16]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('Ru')
page_py = wiki_wiki.page('Литва')
print("Page - Summary: %s" % page_py.summary)

Page - Summary: Литва́ (лит. Lietuva [lʲɪɛtʊˈvɐ]), официальное название — Лито́вская Респу́блика (лит. Lietuvos Respublika) — государство, расположенное в Северной Европе. Площадь — 65 300 км². Протяжённость с севера на юг — 280 км, а с запада на восток — 370 км. Население составляет 2 797 945 человек (июнь, 2022). Занимает 137-е место в мире по численности населения и 121-е по территории. Имеет выход к Балтийскому морю, расположена на его восточном побережье. Береговая линия составляет всего 99 км (наименьший показатель среди государств Балтии). На севере граничит с Латвией, на юго-востоке — с Белоруссией, на юго-западе — с Польшей и Калининградской областью России. По площади и населению является самым крупным государством из стран Балтии.
Столица — Вильнюс. Официальный язык — литовский. Денежная единица — евро.
Восстановление независимости страны провозглашено 11 марта 1990 года. 6 сентября 1991 года Государственный совет СССР признал независимость Литвы. 
Литва — член ООН (1991), О

In [78]:
dictIdTitle = {}
for synset in wn.synsets:
    dictIdTitle[synset.id] = synset.title
    print(synset.id,synset.title)


130542-N ВЫЧИТКА ТЕКСТА
130544-N ГАРПУННАЯ ОХОТА
130545-N ГАРПУН
130547-N ВЫТЕРЕТЬСЯ ОТ НОСКИ
130550-N ВЫТЕЧЬ (О РЕКЕ БРАТЬ НАЧАЛО)
130552-N ОНЕМЕЧИВАНИЕ
130553-N ГЕРМАФРОДИТ
130554-N ГЕРЦ (ЕДИНИЦА ИЗМЕРЕНИЯ)
130555-N ЯПОНСКАЯ ГЕЙША
130558-N ГИДРА (ПРЕСНОВОДНЫЙ ПОЛИП)
130559-N ГИЛЬДИЯ (СОЮЗ, ОБЪЕДИНЕНИЕ)
130560-N ПУЛЕВАЯ ГИЛЬЗА
130561-N ГИПЕРТРОФИЯ ОРГАНА
130562-N ГИПЕРБОЛА (СТИЛИСТИЧЕСКИЙ ПРИЕМ)
130563-N КУПОЛ ЦЕРКВИ
130564-N БИТЬСЯ (ДРАТЬСЯ, СРАЖАТЬСЯ)
130565-N БОЕЦ (УЧАСТНИК БОЯ)
130566-N ДРЕВНИЙ РИМ
130567-N РИМСКИЙ ГЛАДИАТОР
130568-N МИРОВОЙ, ВСЕМИРНЫЙ
130571-N ГЛАЗАСТЫЙ
130572-N ЩЕКАСТЫЙ, С БОЛЬШИМИ ЩЕКАМИ
130574-N КРУГЛОЛИЦЫЙ
130576-N ЛОБАСТЫЙ
130578-N УШАСТЫЙ
130579-N ПРОГЛОТИТЬ ОБИДУ, ОСКОРБЛЕНИЕ
130581-N ДИЧАТЬ, ОДИЧАТЬ
130607-N ГЛУБИНА, ГЛУБОКОЕ МЕСТО
130608-N ГЛУБИНА (ПРОСТРАНСТВО)
130610-N ГЛУШИТЬ, ОГЛУШИТЬ
130611-N ПРЕСЛЕДОВАНИЕ СВОЕЙ ЦЕЛИ
130612-N ЯЙЦЕВИДНАЯ ФОРМА
130613-N ЯЙЦО НАСЕКОМОГО
130625-N ИНОСТРАННОЕ ЮРИДИЧЕСКОЕ ЛИЦО
130626-N ДЛИННОНОГИЙ, С ДЛИННЫМИ НОГАМИ
13062

In [151]:

def add_sign(private:str, candidat:str, dict_lemma_in_text:dict) -> np.array:
    """На вход получаем частное, кандидат(явлюятся id в строковм представление) и информацию о статьях ввикипедии
    выход - полученнный вектор признаков"""
    sign = []
    lemma_private1 = get_normal_form(dictIdTitle[private]).upper()
    lemma_private2 = dictIdTitle[private]
    lemma_candidat1 = get_normal_form(dictIdTitle[candidat]).upper()
    lemma_candidat2 = dictIdTitle[candidat]
    print(lemma_private1,lemma_private2, lemma_candidat1, lemma_candidat2) 
    """в некоторых случаях леммы различаются и нам надо првоерить обе
    пример: домашняя кошка и домашняя кошко"""
    if  lemma_candidat1 not in dict_lemma_in_text and lemma_candidat2 not in dict_lemma_in_text:
        return np.zeros(3)
    else:
        lemma_candidat = ""
        if lemma_candidat1 not in dict_lemma_in_text:
            lemma_candidat = lemma_candidat2
        else:
            lemma_candidat = lemma_candidat1
        sign.append(1)
        if lemma_private1.lower() in dict_lemma_in_text[lemma_candidat].clear_summary or lemma_private2.lower() in dict_lemma_in_text[lemma_candidat].clear_summary:
            sign.append(1)
        else:
            sign.append(0)
        if lemma_private1.lower() in dict_lemma_in_text[lemma_candidat].clear_text or lemma_private1.lower() in dict_lemma_in_text[lemma_candidat].clear_text:
            sign.append(1)
        else:
            sign.append(0)
#         print(dict_lemma_in_text[lemma_candidat].clear_summary)
    return np.array(sign)

In [152]:
# 934-N МЛЕКОПИТАЮЩЕЕ
# 8738-N title: Тигр
add_sign("8738-N","934-N", dict_lemma_in_text)

ТИГР ТИГР МЛЕКОПИТАЮЩЕЕ МЛЕКОПИТАЮЩЕЕ


array([1, 0, 1])

In [153]:
# 110847-N title: Тире lemma: ТИРЕ
# 110846-A ЗНАК ПРЕПИНАНИЯ
add_sign("110847-N","110846-A", dict_lemma_in_text)

ТИРЕ ТИРЕ ЗНАК ПРЕПИНАНИЕ ЗНАК ПРЕПИНАНИЯ


array([1, 0, 1])

In [154]:
#6804-A ДОМАШНЯЯ КОШКA
# 8738-N title: Тигр
add_sign("8738-N","6804-A", dict_lemma_in_text)

ТИГР ТИГР ДОМАШНЯЯ КОШКО ДОМАШНЯЯ КОШКА


array([1, 0, 1])

In [155]:
dict_lemma_in_text[lemma_private].clear_text

NameError: name 'lemma_private' is not defined